# Fetch Data



### Set env variables

assumes that `SOCRATA_APP_TOKEN`, `SOCRATA_API_KEY_ID`, and `SOCRATA_API_KEY_SECRET` are set in `.env`

In [1]:
from dotenv import load_dotenv

load_dotenv()


True

### Load Packages

In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

PACKAGE_PATH = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, PACKAGE_PATH)

from src import fetch
from src import config
from src import forecast
from src import streamlit_app


## Streamlit App

### Low Data For Model

In [ ]:
def load_bundles():
    bundle_mean = forecast.load_bundle(config.MODEL_TIMESTAMP, 'lgb_mean.pkl')
    bundle_90 = forecast.load_bundle(config.MODEL_TIMESTAMP, 'lgb_90.pkl')
    bundle_50 = forecast.load_bundle(config.MODEL_TIMESTAMP, 'lgb_50.pkl')
    bundle_10 = forecast.load_bundle(config.MODEL_TIMESTAMP, 'lgb_10.pkl')

    return bundle_mean, bundle_90, bundle_50, bundle_10

def load_streamlit_data():
    df = pd.read_parquet('s3://hbc-technical-assessment-gk/presentation/DOHMH/streamlit_data.parquet')
    return df

DOHMH Only: ~90 seconds runtime

In [ ]:
# await fetch.fetch_all_service_requests(save = True,
#     columns=config.DOHMH_COLUMNS,
#     additional_filter=config.AGENCY_FILTER
# )

Fetching 2010-01 ...
Fetching 2010-02 ...
Fetching 2010-03 ...
Fetching 2010-04 ...
Fetching 2010-05 ...
Fetching 2010-06 ...
Fetching 2010-07 ...
Fetching 2010-08 ...
Fetching 2010-09 ...
Fetching 2010-10 ...
Fetching 2010-11 ...
Fetching 2010-12 ...
Fetching 2011-01 ...
Fetching 2011-02 ...
Fetching 2011-03 ...
Fetching 2011-04 ...
Fetching 2011-05 ...
Fetching 2011-06 ...
Fetching 2011-07 ...
Fetching 2011-08 ...
Saved s3://hbc-technical-assessment-gk/landing/DOHMH/year=2011/month=01/part-0000.parquet (3,220 rows, 25 columns)
Fetching 2011-09 ...
Saved s3://hbc-technical-assessment-gk/landing/DOHMH/year=2010/month=10/part-0000.parquet (3,728 rows, 25 columns)
Saved s3://hbc-technical-assessment-gk/landing/DOHMH/year=2010/month=02/part-0000.parquet (2,821 rows, 25 columns)
Fetching 2011-10 ...
Fetching 2011-11 ...
Saved s3://hbc-technical-assessment-gk/landing/DOHMH/year=2011/month=03/part-0000.parquet (3,869 rows, 25 columns)
Fetching 2011-12 ...
Saved s3://hbc-technical-assessment-

### Incremental Pull of Data

In [5]:
# await fetch.fetch_current_month_service_requests(save = True)



In [6]:
# await fetch.fetch_current_month_service_requests(save = True,
#     columns=config.DOHDMH_COLUMNS,
#     additional_filter=config.AGENCY_FILTER
# )

### ACS Census Data

In [7]:
df_pop = fetch.fetch_acs_census_population_data(start_year=2013, end_year=2023, save=False)

  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
  - County 005...
  - County 047...
  - County 061...
  - County 081...
  - County 085...
Saved ACS 

### Historical Weather Data

In [8]:
df_weather = fetch.fetch_noaa_weather_data(start_year=2010, end_year=2025, save=False)

Loaded 201001, rows: 155
Loaded 201002, rows: 155
Loaded 201003, rows: 155
Loaded 201004, rows: 155
Loaded 201005, rows: 155
Loaded 201006, rows: 155
Loaded 201007, rows: 155
Loaded 201008, rows: 155
Loaded 201009, rows: 155
Loaded 201010, rows: 155
Loaded 201011, rows: 155
Loaded 201012, rows: 155
Loaded 201101, rows: 155
Loaded 201102, rows: 155
Loaded 201103, rows: 155
Loaded 201104, rows: 155
Loaded 201105, rows: 155
Loaded 201106, rows: 155
Loaded 201107, rows: 155
Loaded 201108, rows: 155
Loaded 201109, rows: 155
Loaded 201110, rows: 155
Loaded 201111, rows: 155
Loaded 201112, rows: 155
Loaded 201201, rows: 155
Loaded 201202, rows: 155
Loaded 201203, rows: 155
Loaded 201204, rows: 155
Loaded 201205, rows: 155
Loaded 201206, rows: 155
Loaded 201207, rows: 155
Loaded 201208, rows: 155
Loaded 201209, rows: 155
Loaded 201210, rows: 155
Loaded 201211, rows: 155
Loaded 201212, rows: 155
Loaded 201301, rows: 155
Loaded 201302, rows: 155
Loaded 201303, rows: 155
Loaded 201304, rows: 155
